In [7]:
from PIL import Image
import os
import cv2
from tqdm import tqdm
import numpy as np

bcso_path = "/home/maximilian/datasets/presentation_sample"

def sort_key(s):
    s = s.strip(".jpg")
    return int(s)

seq_len = 3
images = sorted(os.listdir(bcso_path), key=sort_key)

fps = 5
video_writer = None

for i in tqdm(range(seq_len, len(images))):
    # Load the current image and expand dims to keep it consistent with previous images
    current_image = np.expand_dims(np.array(Image.open(os.path.join(bcso_path, images[i]))), axis=0)
    
    # Load the previous images (seq_len - 1 images) and ensure they are NumPy arrays
    prev_images = np.array([np.array(Image.open(os.path.join(bcso_path, images[i-j]))) for j in range(1, seq_len)])
    
    # Concatenate the images along the width axis (axis=1), so they are side by side
    image_sequence = np.concatenate([current_image] + [np.expand_dims(img, axis=0) for img in prev_images], axis=1)
    
    # Ensure the concatenation happens along the correct axis (width-wise stacking)
    image_sequence = np.squeeze(image_sequence)  # Remove the extra dimension added by expand_dims

    # Initialize the video writer once, using the width and height from the first image sequence
    if video_writer is None:
        h, w, _ = image_sequence.shape
        video_writer = cv2.VideoWriter(
            "/home/maximilian/Master/presentation_video.avi",
            cv2.VideoWriter_fourcc(*'XVID'),  # FourCC codec
            fps,
            (w, h)
        )
    
    # Write the concatenated image sequence to the video
    video_writer.write(cv2.cvtColor(image_sequence, cv2.COLOR_RGB2BGR))

# Release the video writer when done
video_writer.release()

100%|██████████| 314/314 [00:03<00:00, 86.15it/s]
